### **Importing Related Notebooks** 

In [2]:
import import_ipynb
import Data_Understanding

data = Data_Understanding.data

### **Data Preparation**

This section will focus on the process of transforming raw data into ready-to-use data. All of these processes will be examined in each column and, if necessary, delete unnecessary columns or, conversely, add new columns to enrich the insights needed in the Explanaroty Data Analysis section.

##### **Empty Data**

Empty data is usually represented by `Nan`, `None`, and `Null`. When there is empty data, there are many ways to deal with it, including deleting rows or columns containing empty data, filling them with alternative data, or in some cases numeric data, can be filled with mean or median.

In [3]:
import pandas as pd
import numpy as np

null_items = []

for col in data.columns:
    null_items.append([col,data[col].isnull().sum()])

pd.DataFrame(null_items,columns=[
    'Column','Missing Counts'
]).set_index(np.r_[1:17])

,Column,Missing Counts
1,video_id,0
2,trending_date,0
3,title,0
4,channel_title,0
5,category_id,0
6,publish_time,0
7,tags,0
8,views,0
9,likes,0
10,dislikes,0


From here, it is indicated that there is empty data in the `description` column of 570 data while there are no other columns. This needs to be further identified to find out how important the column with empty data is and what to do next.

##### **Data Type Problem**

As the data contained in the `trending_date` column, the date format is not supported for analysis. Therefore, the data type will be converted using the `pd.to_datetime` method in **Pandas**. Likewise, the column `publish_time` will be converted to a similar format.

In [4]:
data[['trending_date','publish_time']].iloc[[1,1000,2000]]

,trending_date,publish_time
1,17.14.11,2017-11-13T07:30:00.000Z
1000,17.19.11,2017-11-18T15:43:11.000Z
2000,17.24.11,2017-11-22T15:00:17.000Z


In [5]:
data['trending_date'] = pd.to_datetime(data['trending_date'],format='%y.%d.%m',utc=False)
data['publish_time'] = pd.to_datetime(pd.to_datetime(data['publish_time']).dt.strftime("%Y-%m-%d"))

display(data[['trending_date','publish_time']].dtypes,data[['trending_date','publish_time']].iloc[[1,1000,2000]])

trending_date    datetime64[ns]
publish_time     datetime64[ns]
dtype: object

,trending_date,publish_time
1,2017-11-14,2017-11-13
1000,2017-11-19,2017-11-18
2000,2017-11-24,2017-11-22


Now the data type in both columns has become `datetime64` with the format YYYY-MM-DD.

In [6]:
data.rename(columns={
    'trending_date':'Trending Date',
    'publish_time':'Publish Date'
},inplace=True)

Now the column that contains date information matches the data type. Then we will change the name of each column sequentially to `Trending Date` and `Publish Date` so that in the following discussion we will use these two names.

##### **Invalid Data**

As previously mentioned, there is a boolean data type containing the values ​​`True` and `False` in the `comments_disabled`, `ratings_disabled` and `video_error_or_removed` fields. This information is not representative, so it needs to be followed up by changing the data according to the intended purpose. Then the column names will also be changed sequentially to `Comments Feature`, `Ratings Feature`, and `Video Availability` so that the column names will be used in further discussion.

In [7]:
for col in ['comments_disabled','ratings_disabled','video_error_or_removed']:
    data.replace({col:{
        True:'Unavailable',
        False:'Available'
    }},inplace=True)

data.rename(columns={
    'comments_disabled':'Comments Feature',
    'ratings_disabled':'Ratings Feature',
    'video_error_or_removed':'Video Availability'
}, inplace=True)

pivot = data.groupby(['Comments Feature','Ratings Feature','Video Availability']).count().reset_index()
pivot['Counts'] = [value for value in data[['Comments Feature','Ratings Feature','Video Availability']].value_counts()]
pivot.pivot_table(index=['Comments Feature','Ratings Feature','Video Availability'],values=['Counts'])

Counts
Comments Feature Ratings Feature Video Availability         
Available        Available       Available           40230.0
                                 Unavailable           527.0
                 Unavailable     Available             106.0
Unavailable      Available       Available              63.0
                 Unavailable     Available              23.0

From the table above, we find that there is a small portion of the video population that is unavailable.

##### **Uninformative Data**

Uninformative data can be interpreted as data that cannot explain the information contained.

In [8]:
data['category_id'].unique()

array([22, 24, 23, 28,  1, 25, 17, 10, 15, 27, 26,  2, 19, 20, 29, 43],
      dtype=int64)

In the `category_id` column, the data contained is still nominally categorical, which, although in the form of a number, has no level or comparison, so it is still considered non-numeric data. However, the data contained is not informative because it is unable to clearly represent the related video. Data values ​​can be changed by matching id data in dictionaty dataset `US_category_id.json`.

In [9]:
data_json = pd.read_json('US_category_id.json')
json_list = []

for i in [json_items for json_items in data_json['items']]:
    json_list.append(i)

pd.set_option('display.max_colwidth',100)
category_df = pd.DataFrame(json_list)[['id','snippet']].rename(columns={'id':'category_id'}).set_index('category_id')
category_df.head()

,snippet
category_id,
1,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Film & Animation', 'assignable': True}"
2,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Autos & Vehicles', 'assignable': True}"
10,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Music', 'assignable': True}"
15,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Pets & Animals', 'assignable': True}"
17,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Sports', 'assignable': True}"


This method can be achieved with the help of library **re** (regular expression). The goal is to find a condition where all data in the `category_id` column is no longer a number. When this condition is reached, the iteration process and data substitution can be stopped so that the process does not run forever even though in fact the process that is executed requires quite a long time.

In [10]:
import re

for category_id in data['category_id']:
    data.loc[data['category_id']==category_id,'category_id'] = category_df['snippet'].loc[str(category_id)].setdefault('title')
    if not re.match(r'(?=[0-9])',str(data['category_id'].unique()[-1])):
        break

data['category_id'].unique()

array(['People & Blogs', 'Entertainment', 'Comedy',
       'Science & Technology', 'Film & Animation', 'News & Politics',
       'Sports', 'Music', 'Pets & Animals', 'Education', 'Howto & Style',
       'Autos & Vehicles', 'Travel & Events', 'Gaming',
       'Nonprofits & Activism', 'Shows'], dtype=object)

After the substitution process is complete, the `category_id` column is filled with clear information regarding the categorization of the related video. In addition, the column name will also be changed to `Category` so that the new column name will continue to be used in subsequent discussions.

In [11]:
data.rename(columns={'category_id':'Category'},inplace=True)

##### **Duplicate Data**

As previously mentioned, the `video_id` data is a unique code embedded in a video. **YouTube** uses the Base 64 system which involves uppercase and lowercase letters, numbers, and hyphens ("-") and underscores ("_") which consists of 11 characters.

In [12]:
for vid_id in data['video_id']:
    if not re.match(r'^[A-Za-z0-9-_]+$',vid_id) or len(vid_id) != 11:
        print('There are data with inappropriate characters')
else:
    print('There no data with characters excluding uppercase and lowercase letters, numbers,'
          '\nand hyphens ("-") and underscores ("_") which consists of 11 characters.')

There no data with characters excluding uppercase and lowercase letters, numbers,
and hyphens ("-") and underscores ("_") which consists of 11 characters.


Although the `video_id` for each video is unique, in reality, it represents a different entity in the dataset. This is because, at certain times, the video owner makes changes to several video attributes such as `title`, `channel_title`, `category_id`, `tags`, `Comments Feature`, and `Ratings Feature`.

To ensure that each unique data in the `video_id` column does not represent more than one video, you can check the number of unique data in the `Publish Date` column. Because it is very clear that a video uploaded at one time will have different unique data from other videos even though it was uploaded on the same day. If more than one data is found in the `Publish Date` column, then it is certain that it is indicated as a duplicate. This can be done by grouping using `groupby()` method in the `video_id` column and aggregating with `agg()` method in the `Publish Date` column.

In [13]:
vid_id_df = data.groupby('video_id').agg(Total_Publish_Date=('Publish Date','nunique'))
pd.DataFrame(vid_id_df.value_counts())

,count
Total_Publish_Date,
1,6349
2,1
3,1


From the process above, an event was found where there was 2 unique data from the `video_id` column which indicated duplicates. In this case, duplicate data can be removed as part of the guarantee that all data `video_id` is unique. However further checks are required to know the intent of the event on this dataset.

In [14]:
display(
    vid_id_df[vid_id_df['Total_Publish_Date']!=1],
    pd.DataFrame(
        data[(data['video_id']=='kZete48ZtsY') | (data['video_id']=='ogfYd705cRs')]\
                [['video_id','title','channel_title','Publish Date','Video Availability']].value_counts(sort=False)
    )
)

,Total_Publish_Date
video_id,
kZete48ZtsY,3
ogfYd705cRs,2


count
video_id    title                                            channel_title          Publish Date Video Availability       
kZete48ZtsY Deleted video                                    DaHoopSpot Productions 2017-12-16   Unavailable             2
                                                             Midnight Video         2018-01-29   Unavailable             1
            Roger Federer 20th Grand Slam Victory Tribute    ATPWorldTour           2018-01-28   Available               2
            Roger Federer's  20th Grand Slam Victory Tribute ATPWorldTour           2018-01-28   Available               1
ogfYd705cRs Google I/O'18: Google Keynote                    Google Developers      2018-04-24   Available               1
            Keynote (Google I/O '18)                         Google Developers      2018-05-10   Available              13

The findings above explain that:
1. As can be seen in the `n_unique_date` column, the unique data 'kZete48ZtsY' is used by three different videos. However, this is not a problem because the data is not used simultaneously at one time. In other words, **YouTube** creates a reusable system to manage unique data from the `video_id` column.

2. A unique data `video_id` can be reused on condition that the previously linked video is unavailable on the **YouTube** service.
3. In certain cases such as the unique data 'ogfYd705cRs', there are two videos released at different times but use the same unique data even though one of the videos has not been deleted. This indicates duplication even though launched by the same channel. To overcome this, we will perform a transformation on the `Publish Date` column using the earliest data.

In [15]:
data.loc[(data['channel_title']=='Google Developers') & 
         (data['video_id']=='ogfYd705cRs') & 
         (data['Publish Date']!='2018-04-24'),
         'Publish Date'] = data[(data['channel_title']=='Google Developers') & (data['video_id']=='ogfYd705cRs')]['Publish Date'].min()

pd.DataFrame(
    data[data['video_id']=='ogfYd705cRs'][[
        'video_id','title','channel_title','Publish Date','Video Availability'
    ]].value_counts(sort=False)
)

count
video_id    title                         channel_title     Publish Date Video Availability       
ogfYd705cRs Google I/O'18: Google Keynote Google Developers 2018-04-24   Available               1
            Keynote (Google I/O '18)      Google Developers 2018-04-24   Available              13

The `Publish Date` data from the unique data 'ogfYd705cRs' is uniform so there is no indication of any further duplication. Then we will change the column name `video_id` to `Video ID` so that the new name will be used in the following discussion.

In [16]:
data.rename(columns={'video_id':'Video ID'},inplace=True)

##### **Incosys Date**

There are no clear rules regarding the video title naming system in the `title` column. It's just that there is a limit regarding the number of characters displayed, which is as many as 100 characters, including spaces. If it exceeds the limit, it will be truncated at the end of a title.

In [17]:
title_list = []

for i in data['title']:
    title_list.append(len(i))

print('The longest video title is',"'"+data.iloc[np.array(title_list).argmax()]['title']+"'.",
      '\nThe shortest video title is',"'"+data.iloc[np.array(title_list).argmin()]['title']+"'.")

The longest video title is 'LeBron James admits he was ripping Phil Jackson and thinks DeShaun Watson should be the Browns qu...'. 
The shortest video title is '435'.


Likewise for the channel name in the `channel_title` column, no definite rule was found in the data.

In [18]:
channel_list = []

for i in data['channel_title']:
    channel_list.append(len(i))

print('The longest channel name is',"'"+data.iloc[np.array(channel_list).argmax()]['channel_title']+"'.",
      '\nThe shortest channel name is',"'"+data.iloc[np.array(channel_list).argmin()]['channel_title']+"'.")

The longest channel name is '42Fab - Metalworking and Multi-Medium Fabrication'. 
The shortest channel name is 'GQ'.


In [19]:
pd.DataFrame(data[['title','channel_title']].value_counts().head())

,,count
title,channel_title,
WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,30
Why I'm So Scared (being myself and crying too much),grav3yardgirl,29
Mission: Impossible - Fallout (2018) - Official Trailer - Paramount Pictures,Paramount Pictures,29
"The ULTIMATE $30,000 Gaming PC Setup",Unbox Therapy,29
YoungBoy Never Broke Again Goes Sneaker Shopping With Complex,Complex,29


Even though the data in the two columns is not quite consistent in writing, this is understandable as long as this is a freedom given by **YouTube** as a service provider to video owners. However, due to changes in information made over time, the data will be transformed based on the maximum data from the `Trending Date` column. In other words, the data in both columns will be uniform using the most updated data attributes.

In [20]:
updated = data[['Video ID','title','channel_title','Trending Date']].groupby('Video ID')[[
    'title','channel_title','Trending Date'
]].transform('max')

In [21]:
data['title'] = updated['title']
data['channel_title'] = updated['channel_title']
data.rename(columns={
    'title':'Title',
    'channel_title':'Channel'
},inplace=True)

Then the column names will be changed sequentially to `Title` and `Channel` so that these two names will be used in the following discussion.

##### **Feature Engineering**

The `tags` column contains information about the context embedded by the video owner to help potential viewers find their videos.

In [22]:
pd.set_option('display.max_colwidth',200)
pd.DataFrame(data['tags'].value_counts().head(3)).rename(columns={'tags':'Occurences'}).reset_index().rename(columns={'index':'tags'})

,tags,count
0,[none],1535
1,"ABC|""americanidol""|""idol""|""american idol""|""ryan""|""seacrest""|""ryan seacrest""|""katy""|""perry""|""katy perry""|""luke""|""bryan""|""luke bryan""|""lionel""|""richie""|""lionel richie""|""season 16""|""american idol XVI...",87
2,"Jacksfilms|""Jack Douglass""|""YGS""|""YGS 100""|""YGS 50""|""The Best of Your Grammar Sucks""|""Your Grammar Sucks""|""YIAY""|""Yesterday I Asked You""|""Fidget Spinners""|""Emoji Movie""|""Kermit Sings""|""JackAsk""|""J...",80


The information contained does not have standard rules and tends to be inconsistent. However, the data has a vertical bar ("|") character as a separator between hashtags. In addition, there is data '[none]' which indicates no hashtags are used. From here, feature engineering is done to calculate how many hashtags are used in each video.

In [23]:
num_tags = []

for tags in data['tags']:
    if re.match(r'^\[none]+$',tags):
        num_tags.append(0)
        continue
    num_tags.append(len(tags.split('|')))

data['Total Tags'] = num_tags
data.rename(columns={'tags':'Tags'},inplace=True)

The column for the number of hashtags used will be named `Total Tags`. Then, the `tags` column itself will be renamed to `Tags` so that the new name will be used in further discussion.

In [24]:
pd.DataFrame(data['Total Tags'].value_counts().head().reset_index().rename(columns={
    'index':'Tags Used',
    'Total Tags':'Count'
}))

,Count,count
0,0,1535
1,10,1447
2,4,1405
3,7,1388
4,13,1299


The number of videos that don't use hashtags is in first place. Next, other features will be extracted from the `Tags` column, namely common tags which are classified based on the `Video ID` column.

In [25]:
data.pivot_table(
    index=['Video ID','Tags'],
    values=['Total Tags'],
    aggfunc='count').head().rename(columns={'Total Tags':'Occurences'}
)

,,Occurences
Video ID,Tags,
-0CMnp02rNY,"ellen|""ellen degeneres""|""the ellen show""|""ellentube""|""ellen audience""|""season 15 episode 165""|""mindy kaling""|""mindy kaling baby""|""oprah""|""mindy""|""kaling""|""mindy kaling the office""|""mindy kaling a wrinkle in time""|""mindy kaling and b.j. novak""|""katherine""|""oprahs house""|""ellen fans""|""ellen tickets""|""season 15""|""daughter""|""mindy kaling daughter""|""bj novak""|""baby daddy""|""ocean's 8""|""oceans 8 movie""|""the office""|""interview""|""new""|""funny""|""hilarious""|""sandra bullock""|""anne hathaway""|""wrinkle in time""",6
-0NYY8cqdiQ,"megan mullally|""megan""|""mullally""|""will and grace""|""karen on will and grace""|""actress""|""nick offerman""|""Ellen""|""degeneres""|""ellen degeneres""|""the ellen show""|""ellen fans""|""ellen tickets""|""ellentube""|""ellen audience""|""will""|""grace""|""karen""|""roomats""|""funny""|""interview""|""nick""|""offerman""|""wedding""|""marriage""|""roomates""|""story""|""hilarious""|""bobs burgers""|""emmy""|""hobby""|""females""|""naive""",1
-1Hm41N0dUs,"jimmy|""jimmy kimmel""|""jimmy kimmel live""|""late night""|""talk show""|""funny""|""comedic""|""comedy""|""clip""|""comedian""|""mean tweets""|""Benedict Cumberbatch""|""Don Cheadle""|""Elizabeth Olsen""|""winston duke""|""avengers: infinity war""|""marvel""|""marvel cinematic universe""|""scarlet witch""|""war machine""|""doctor strange""|""m'baku""|""black panther""",3
-1yT-K3c6YI,"youtube quiz|""youtuber quiz""|""truth or dare""|""exposed""|""youtube crush""|""molly burk""|""collab""|""collaboration""",4
-2RVw2_QyxQ,"Chess|""Saint Louis""|""Club""",3


From here, common tags will collect any hashtags from a video that are also used by other videos, both when adding or removing related videos. The first step is to take each data in the `Tags` column and clean it from vertical bar ("|") and comma (",") characters. Next, a dictionary list will be created consisting of data in the `Video ID` column as keys and the `Tags` column as values.

In [26]:
common_list = []

for i in data.pivot_table(index=['Video ID','Tags'],values=['Total Tags']).reset_index()[['Video ID','Tags']].values:
    scrub_dirt = re.sub(r'[|]',',',i[1])
    common_list.append({i[0]:re.sub(r'["]','',scrub_dirt).split(',')})

Each dictionary will be temporarily accommodated in dictionary list with the name `common_list`. Then, it will search for intersection of a list with all other lists so that common tags from each related video are found. This process takes a long time depending on the number of hashtags in a list and the number of dictionaries of the dictionary list itself.

In [27]:
from functools import reduce

common = []

for i in common_list:
    stacked_tags = []
    for n in common_list:
        intersect_tags = []
        if (list(i.keys()).pop() != list(n.keys()).pop()) and '[none]' not in list(i.values()).pop():
            intersection = [list(i.values()).pop(),list(n.values()).pop()]
            intersect_tags.append(reduce(lambda x,y: set(x) & set(y),intersection))
        try:
            stacked_tags.append(reduce(lambda x,y: set(x) & set(y),intersect_tags))
        except:
            stacked_tags.append(set())
    common.append(list(set().union(*stacked_tags)))

tags_df = data.pivot_table(index=['Video ID','Tags'],values=['Total Tags']).reset_index()[['Video ID','Tags']]
tags_df['Common'] = pd.Series(common)
tags_df.head()

,Video ID,Tags,Common
0,-0CMnp02rNY,"ellen|""ellen degeneres""|""the ellen show""|""ellentube""|""ellen audience""|""season 15 episode 165""|""mindy kaling""|""mindy kaling baby""|""oprah""|""mindy""|""kaling""|""mindy kaling the office""|""mindy kaling a ...","[hilarious, new, ellen tickets, baby daddy, funny, ellen fans, daughter, mindy kaling, interview, ellen audience, the ellen show, season 15, katherine, sandra bullock, ellentube, the office, ellen..."
1,-0NYY8cqdiQ,"megan mullally|""megan""|""mullally""|""will and grace""|""karen on will and grace""|""actress""|""nick offerman""|""Ellen""|""degeneres""|""ellen degeneres""|""the ellen show""|""ellen fans""|""ellen tickets""|""ellentub...","[hilarious, will and grace, ellen tickets, funny, ellen fans, will, wedding, hobby, story, degeneres, interview, ellen audience, the ellen show, actress, Ellen, bobs burgers, nick, ellentube, grac..."
2,-1Hm41N0dUs,"jimmy|""jimmy kimmel""|""jimmy kimmel live""|""late night""|""talk show""|""funny""|""comedic""|""comedy""|""clip""|""comedian""|""mean tweets""|""Benedict Cumberbatch""|""Don Cheadle""|""Elizabeth Olsen""|""winston duke""|""...","[talk show, marvel, comedic, funny, comedy, late night, avengers: infinity war, jimmy, doctor strange, m'baku, Don Cheadle, Elizabeth Olsen, comedian, jimmy kimmel, Benedict Cumberbatch, mean twee..."
3,-1yT-K3c6YI,"youtube quiz|""youtuber quiz""|""truth or dare""|""exposed""|""youtube crush""|""molly burk""|""collab""|""collaboration""","[truth or dare, collaboration, collab, exposed]"
4,-2RVw2_QyxQ,"Chess|""Saint Louis""|""Club""","[Chess, Saint Louis, Club]"


Then the data in the `Common` column of dataframe `tags_df` will be transformed into the `Common` column of dataframe `data` based on the classification of the `Video ID` and ` Tags`. Therefore, a new temporary column named `temp_key` will be created on each dataframe.

In [28]:
(data['temp_key'],tags_df['temp_key']) = (
    data['Video ID'].astype(str) + data['Tags'].astype(str),
    tags_df['Video ID'].astype(str) + tags_df['Tags'].astype(str)
)

data['Common'] = data['temp_key'].map(tags_df.set_index('temp_key')['Common'])
data.drop('temp_key',axis=1,inplace=True)

After the transformation process is complete, the `temp_key` field in dataframe `data` needs to be deleted. Then a new feature will be created in the column `Common Rate (%)` to measure the percentage of the number of common tags of a video based on the total hashtags used.

In [29]:
def imputed_common_rate(rate):
    try:
        return (len(rate[0])/rate[1])*100
    except ZeroDivisionError:
        return float(0)

data['Common Rate (%)'] = data[['Common','Total Tags']].apply(imputed_common_rate,axis=1)
data['Common Rate (%)'] = data[['Common Rate (%)']].apply(lambda row: row.map('{:.2f}'.format),axis=1).astype('float64')

Furthermore, from the `Common Rate (%)` column, similarity degree of a video and other videos can be found by looking at the number of hashtags used together. The goal is to find out the similarity level based on the context used. A new column named `Similarity` can be created based on the data in the `Common Rate (%)` column with the following conditions:

1. If it is equal to 100 %, then the label is 'Identical'.

1. If more than 75%, then the label is 'Similar'.
1. If between 50% to 75%, then the label is 'Alike'.
1. If between 25% to 50%, then the label is 'Somewhat'.
1. If between 0 % to 25 %, then the label is 'Slight'.
1. If not at all, then the label is 'Distinct'.

In [30]:
def imputed_similarity(rate):
    if rate == 100:
        return 'Identical'
    elif rate >= 75 and rate <100:
        return 'Similar'
    elif rate >= 50 and rate < 75:
        return 'Alike'
    elif rate >= 25 and rate < 50:
        return 'Somewhat'
    elif rate > 0 and rate < 25:
        return 'Slight'
    else:
        return 'Distinct'

data['Similarity'] = data['Common Rate (%)'].apply(imputed_similarity)
pd.DataFrame(data['Similarity'].value_counts()).rename(columns={'Similarity':'Occurences'})

,count
Similarity,
Alike,12774
Similar,11265
Somewhat,7005
Identical,4506
Slight,2978
Distinct,2421


Next, the `Tags` column will be cleaned of inappropriate characters and the data type will be changed to list.

In [31]:
def cleaning_hashtags(tags):
    if tags == '[none]':
        return ['']
    scrub_dirt = re.sub(r'[|]',',',tags)
    return re.sub(r'["]','',scrub_dirt).split(',')

data['Tags'] = data['Tags'].apply(cleaning_hashtags)
pd.set_option('display.max_colwidth',150)
data[['Tags']].head()

,Tags
0,[SHANtell martin]
1,"[last week tonight trump presidency, last week tonight donald trump, john oliver trump, donald trump]"
2,"[racist superman, rudy, mancuso, king, bach, racist, superman, love, rudy mancuso poo bear black white official music video, iphone x by pineapple..."
3,"[rhett and link, gmm, good mythical morning, rhett and link good mythical morning, good mythical morning rhett and link, mythical morning, Season ..."
4,"[ryan, higa, higatv, nigahiga, i dare you, idy, rhpc, dares, no truth, comments, comedy, funny, stupid, fail]"


The four columns `views`, `likes`, `dislikes` and `comment_count` contain information about the cumulative number of occurrences of related videos. The number tends to increase from day to day as long as the video is trending. Descriptive statistics regarding these columns can be seen as follows.

In [32]:
numeric = data[['views','likes','dislikes','comment_count']].describe()
numeric.iloc[1:3] = numeric.iloc[1:3].apply(lambda row: row.map('{:.2f}'.format),axis=1)
numeric

,views,likes,dislikes,comment_count
count,40949.0,40949.0,40949.0,40949.0
mean,2360784.64,74266.70,3711.40,8446.80
std,7394113.76,228885.34,29029.71,37430.49
min,549.0,0.0,0.0,0.0
25%,242329.0,5424.0,202.0,614.0
50%,681861.0,18091.0,631.0,1856.0
75%,1823157.0,55417.0,1938.0,5755.0
max,225211923.0,5613827.0,1674420.0,1361580.0


The amount of data listed for each column is 40949 data along with information on the distribution and data center points. From this information we can perform a feature engineering process. Among them is to calculate the engagement rate obtained from dividing the number of likes and comments to the number of views and save the data in the `Engagement Rate (%)` column.

In [33]:
data['Engagement Rate (%)'] = data[['views','likes','comment_count']].apply(lambda x: ((x[1]+x[2])/x[0])*100,axis=1)
data['Engagement Rate (%)'] = data[['Engagement Rate (%)']].apply(lambda row: row.map('{:.2f}'.format),axis=1).astype('float64')

Then a new column named `Quality` will also be created as a representative value of the `Engagement Rate (%)` column with the following provisions:
1. If more than or equal to 15%, then the quality is 'Excellent'.

2. If between 3% to 15%, then the quality is 'Good'.
3. If it is between 1% and 3%, then the quality is 'Fair'.
4. If it is below 1%, then the quality is 'Poor'.

In [34]:
def imputed(rate):
    return ('Excellent' if rate >= 15.0 else 'Good' if rate < 15.0 and rate >= 3.0 
            else 'Fair' if rate < 3.0 and rate >= 1.0 else 'Poor')

data['Quality'] = data['Engagement Rate (%)'].apply(imputed)

Next, the column names `views`, `likes`, `dislikes`, and `comment_count` will be edited to be more tidy so that the new names will be used in further discussion.

In [35]:
data = data.rename(columns={
    'views':'Views',
    'likes':'Likes',
    'dislikes':'Dislikes',
    'comment_count':'Comments'
})

##### **Dropping Unnecessary**

After the necessary features have been created, the next step is to clean dataset from unnecessary columns and it would be much better if column reordering was done to facilitate the data analysis process. The columns to be removed include `thumbnail_link` and `description` for the following reasons:
1. The column `thumbnail_link` is a link from a related video which information can identify whether a `Video ID` is unique or not. Since `Video ID` has been confirmed not to be duplicated in the previous process, this column is no longer needed.

1. The `description` column does not have strong enough reasons to be used as a consideration in measuring the popularity metrics of a video. The information reviewed is only promotional, informational, collaborative, solicitation, explanatory in nature, and can even be left blank. Therefore, this column will also be deleted.

In [36]:
pd.set_option('display.max_colwidth',50)
data.drop(['thumbnail_link','description'],axis=1)
data = data.reindex(columns=[
    'Video ID','Title','Channel','Category','Publish Date','Trending Date','Views','Likes','Comments','Engagement Rate (%)',
    'Quality','Dislikes','Tags','Total Tags','Common','Common Rate (%)','Similarity','Ratings Feature','Comments Feature',
    'Video Availability'
])

data.head()

,Video ID,Title,Channel,Category,Publish Date,Trending Date,Views,Likes,Comments,Engagement Rate (%),Quality,Dislikes,Tags,Total Tags,Common,Common Rate (%),Similarity,Ratings Feature,Comments Feature,Video Availability
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,People & Blogs,2017-11-13,2017-11-14,748374,57527,15954,9.82,Good,2966,[SHANtell martin],1,[],0.00,Distinct,Available,Available,Available
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,2017-11-13,2017-11-14,2418783,97185,12703,4.54,Good,6146,"[last week tonight trump presidency, last week...",4,"[john oliver trump, donald trump]",50.00,Alike,Available,Available,Available
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12,2017-11-14,3191434,146033,8181,4.83,Good,5339,"[racist superman, rudy, mancuso, king, bach, r...",23,"[racist superman, racist, anwar, rudy, anitta,...",91.30,Similar,Available,Available,Available
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,Entertainment,2017-11-13,2017-11-14,343168,10172,2146,3.59,Good,666,"[rhett and link, gmm, good mythical morning, r...",27,"[music, rhett and link good mythical morning, ...",55.56,Alike,Available,Available,Available
4,d380meD0W0M,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12,2017-11-14,2095731,132235,17518,7.15,Good,1989,"[ryan, higa, higatv, nigahiga, i dare you, idy...",14,"[stupid, fail, higatv, funny, comedy, ryan, ni...",64.29,Alike,Available,Available,Available


In [37]:
print(
    'Dataframe Infographic: \n\n1. Dataframe has {} rows dan {} columns.'.format(data.shape[0],data.shape[1]),
    '\n2. There are {} data types, those are {}.'.format(
        len(set(str(i) for i in data.dtypes.values)),
        ', '.join(list(set(str(i) for i in data.dtypes.values)))
    )
)

infografis = pd.DataFrame(data.dtypes).reset_index().rename(columns={'index':'Column',0:'Data Type'}).set_index(np.r_[1:21])
infografis['Missing Counts'] = data.isnull().sum().values
infografis

Dataframe Infographic: 

1. Dataframe has 40949 rows dan 20 columns. 
2. There are 4 data types, those are object, float64, int64, datetime64[ns].


,Column,Data Type,Missing Counts
1,Video ID,object,0
2,Title,object,0
3,Channel,object,0
4,Category,object,0
5,Publish Date,datetime64[ns],0
6,Trending Date,datetime64[ns],0
7,Views,int64,0
8,Likes,int64,0
9,Comments,int64,0
10,Engagement Rate (%),float64,0


After all Data Preparation processes have been carried out, at this stage the data is ready to be processed in the next process, namely Explanatory Data Analysis. Before that, let's save the prepared data into a new file called `USvideos_clean.csv`.

In [38]:
data.to_csv('USvideos_clean.csv',index=False)